<a href="https://colab.research.google.com/github/vengie/AML-3104--Neural-Networks-and-Deep-Learning-Assignment-1/blob/main/Improved_Music_Composition_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation, BatchNormalization
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [5]:
SEQUENCE_LENGTH = 100
EPOCHS = 200
BATCH_SIZE = 128

In [6]:
def train_network():
    """Train a Neural Network to generate music."""
    notes = get_notes()
    n_vocab = len(set(notes))
    network_input, network_output = prepare_sequences(notes, n_vocab)
    model = create_network(network_input, n_vocab)
    train(model, network_input, network_output)

In [7]:
def get_notes():
    """Get all the notes and chords from the midi files."""
    notes = []
    for file in glob.glob("/content/midi_songs/*.mid"):
        midi = converter.parse(file)
        print(f"Parsing {file}")
        try:
            parts = instrument.partitionByInstrument(midi)
            notes_to_parse = parts.parts[0].recurse()
        except:
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('/content/data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
    return notes

In [8]:
def prepare_sequences(notes, n_vocab):
    """Prepare sequences used by the Neural Network."""
    pitchnames = sorted(set(notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input, network_output = [], []
    for i in range(0, len(notes) - SEQUENCE_LENGTH, 1):
        in_seq = notes[i:i + SEQUENCE_LENGTH]
        out_seq = notes[i + SEQUENCE_LENGTH]
        network_input.append([note_to_int[char] for char in in_seq])
        network_output.append(note_to_int[out_seq])

    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, SEQUENCE_LENGTH, 1))
    network_input = network_input / float(n_vocab)
    network_output = np_utils.to_categorical(network_output)
    return network_input, network_output

In [9]:
def create_network(network_input, n_vocab):
    """Create the structure of the neural network."""
    model = Sequential()
    model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), recurrent_dropout=0.3, return_sequences=True))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3))
    model.add(LSTM(512))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

In [10]:
def train(model, network_input, network_output):
    """Train the neural network."""
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]
    model.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks_list)

In [ ]:
if __name__ == '__main__':
    train_network()

Parsing /content/midi_songs/figaro.mid
Parsing /content/midi_songs/Life_Stream.mid
Parsing /content/midi_songs/Ff7-Cinco.mid
Parsing /content/midi_songs/Ff7-Jenova_Absolute.mid
Parsing /content/midi_songs/FF3_Battle_(Piano).mid
Parsing /content/midi_songs/Finalfantasy5gilgameshp.mid


/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=6, channel=None, data=b'Pok\xe9mon: Elite Four (Piano)'>; getting generic Instrument
  warnings.warn(


Parsing /content/midi_songs/pkelite4.mid
Parsing /content/midi_songs/waltz_de_choco.mid
Parsing /content/midi_songs/Eternal_Harvest.mid
Parsing /content/midi_songs/thoughts.mid
Parsing /content/midi_songs/rufus.mid
Parsing /content/midi_songs/ultros.mid
Parsing /content/midi_songs/FF8_Shuffle_or_boogie_pc.mid
Parsing /content/midi_songs/Kingdom_Hearts_Traverse_Town.mid
Parsing /content/midi_songs/Finalfantasy6fanfarecomplete.mid
Parsing /content/midi_songs/FFX_-_Ending_Theme_(Piano_Version)_-_by_Angel_FF.mid
Parsing /content/midi_songs/JENOVA.mid
Parsing /content/midi_songs/Fyw_piano.mid
Parsing /content/midi_songs/ff1battp.mid
Parsing /content/midi_songs/bcm.mid
Parsing /content/midi_songs/decisive.mid
Parsing /content/midi_songs/AT.mid
Parsing /content/midi_songs/tifap.mid
Parsing /content/midi_songs/dayafter.mid


/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Martin Rosok Copyright \xa9 1998'>; getting generic Instrument
  warnings.warn(


Parsing /content/midi_songs/ff7themep.mid
Parsing /content/midi_songs/sera_.mid
Parsing /content/midi_songs/FF4.mid
Parsing /content/midi_songs/balamb.mid
Parsing /content/midi_songs/ff4-fight1.mid
Parsing /content/midi_songs/dontbeafraid.mid
Parsing /content/midi_songs/ahead_on_our_way_piano.mid
Parsing /content/midi_songs/Suteki_Da_Ne_(Piano_Version).mid
Parsing /content/midi_songs/Still_Alive-1.mid
Parsing /content/midi_songs/Rachel_Piano_tempofix.mid
Parsing /content/midi_songs/ff4-airship.mid
Parsing /content/midi_songs/ff11_awakening_piano.mid
Parsing /content/midi_songs/path_of_repentance.mid
Parsing /content/midi_songs/Zelda_Overworld.mid
Parsing /content/midi_songs/Kingdom_Hearts_Dearly_Beloved.mid
Parsing /content/midi_songs/BlueStone_LastDungeon.mid
Parsing /content/midi_songs/great_war.mid
Parsing /content/midi_songs/Oppressed.mid
Parsing /content/midi_songs/fortresscondor.mid
Parsing /content/midi_songs/Rydia_pc.mid
Parsing /content/midi_songs/DOS.mid
Parsing /content/midi